In [2]:
import pandas as pd

In [5]:
df = pd.read_parquet('data/base/Slovakia_GDP_(USD).parquet')
df = df.set_index('Year')
df.index = pd.to_datetime(df.index, format='%Y')
df = df.drop('Indicator', axis = 1)
df = df.dropna()
df.head()

,Value
Year,
1990-01-01,1.291505e+10
1991-01-01,1.445992e+10
1992-01-01,1.569933e+10
1993-01-01,1.673797e+10
1994-01-01,2.042814e+10


### ACTUAL

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense , Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid


def save_plot(train, test_y, predictions, country, indicator, model_name):
    """Function to save the plot in both Indicators and Countries folders."""

    model_colors = {
    "ARIMA": "blue",
    "Holt_Winters": "yellow",
    "LSTM": "black",
    "XGBoost": "pink",
    "Prophet": "brown"
}

    # Plotting predicted vs actual
    plt.figure(figsize=(10, 6))
    if model_name == "Prophet":
        plt.plot(train['ds'], train['y'], label='Train Data', color='green', linestyle='--')
        plt.plot(test_y['ds'], test_y['y'], label='Actual', color='red', linestyle='--')
        plt.plot(test_y['ds'], predictions, label=f'Predicted({model_name})', color=f'{model_colors["Prophet"]}', 
                 linestyle='-', marker='o')
    else:
        plt.plot(train.index, train, label='Train Data', color='green', linestyle='--')
        plt.plot(test_y.index, test_y, label='Actual', color='red', linestyle='--')
        plt.plot(test_y.index, predictions, label=f'Predicted({model_name})', color=f'{model_colors[model_name]}', 
                 linestyle='-', marker='o')
    

    
    plt.title(f'Predicted({model_name}) vs Actual for {country} - {indicator}')
    plt.xlabel('Year')
    plt.ylabel('Value')
    plt.legend()

    # Create subfolder for the indicator if it doesn't exist
    indicator_folder = os.path.join('images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    # Save the plot in the Indicators folder with dynamic model name
    plot_filename_indicator = os.path.join(indicator_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_indicator)

    # Create subfolder for the country if it doesn't exist
    country_folder = os.path.join('images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    
    # Save the same plot in the Countries folder with dynamic model name
    plot_filename_country = os.path.join(country_folder, f'{model_name}_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
    plt.savefig(plot_filename_country)

    plt.close()

def train_xgboost(train_x, train_y, test_x, test_y, country, indicator):
    params_dir = os.path.join("best_params", indicator)
    params_file = os.path.join(params_dir, f"XGBoost_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define the parameter grid for GridSearchCV
        param_grid = {
            'n_estimators': [50, 100, 600, 700, 800, 900, 1000],
            'learning_rate': [0.01, 0.05],
            'max_depth': [3, 4, 5, 6]
        }

        # Initialize the XGBoost regressor
        model = xgb.XGBRegressor(objective='reg:squarederror')

        # Perform GridSearchCV to find the best hyperparameters
        grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1, verbose=0)
        grid_search.fit(train_x, train_y)

        # Get the best parameters and save them to JSON
        best_params = grid_search.best_params_
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)

    # Train the best model using the loaded or selected parameters
    best_model = xgb.XGBRegressor(
        n_estimators=best_params['n_estimators'],
        learning_rate=best_params['learning_rate'],
        max_depth=best_params['max_depth'],
        objective='reg:squarederror'
    )

    # Perform Recursive Feature Elimination (RFE)
    top_n_features = 4
    selector = RFE(estimator=best_model, n_features_to_select=top_n_features)
    selector = selector.fit(train_x, train_y)

    # Get selected feature names
    selected_features = train_x.columns[selector.support_]

    # Save selected features to JSON
    features_file = os.path.join(params_dir, f"SelectedFeatures_{country}.json")
    with open(features_file, "w") as f:
        json.dump(selected_features.tolist(), f, indent=4)

    # Transform data based on selected features
    selected_train_x = selector.transform(train_x)
    selected_test_x = selector.transform(test_x)

    # Train the model again with the selected features
    best_model.fit(selected_train_x, train_y)

    # Make predictions on the test set using the selected features
    predictions = best_model.predict(selected_test_x)

    # Save the plot
    save_plot(train_y, test_y, predictions, country, indicator, model_name="XGBoost")

    # Calculate the RMSE on the test set
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions


def train_arima(train_y, test_y, country, indicator):
    # Ensure the time index is correctly set
    train_y.index = pd.date_range(start=train_y.index[0], periods=len(train_y), freq='YE')  # Assuming yearly data
    test_y.index = pd.date_range(start=test_y.index[0], periods=len(test_y), freq='YE')

    # Define paths for saving/loading best parameters
    params_dir = os.path.join("best_params", indicator)
    params_file = os.path.join(params_dir, f"ARIMA_{country}.json")
    best_params = None

    # Check if the best parameters JSON file exists
    if os.path.exists(params_file):
        with open(params_file, "r") as f:
            best_params = json.load(f)
        print(f"Loaded best parameters for {country} - {indicator} from {params_file}: {best_params}")
    else:
        # Run the grid search if no parameters file exists
        print(f"No pre-existing parameters for {country} - {indicator}. Running grid search.")
        best_rmse = float('inf')
        best_order = None
        best_predictions = None

        # Perform grid search over ARIMA orders (p, d, q)
        for p in range(15):
            for d in range(6):
                for q in range(10):
                    try:
                        # Fit the ARIMA model
                        model = ARIMA(train_y, order=(p, d, q))
                        model_fit = model.fit()

                        # Forecast and calculate RMSE
                        predictions = model_fit.forecast(steps=len(test_y))
                        rmse = np.sqrt(mean_squared_error(test_y, predictions))

                        # Track the best parameters
                        if rmse < best_rmse:
                            best_rmse = rmse
                            best_order = (p, d, q)
                            best_predictions = predictions
                    except Exception as e:
                        print(f"Error with parameters: p={p}, d={d}, q={q}. Error: {e}")
                        continue

        # Save the best parameters to JSON for future use
        if best_order is not None:
            best_params = {
                "p": best_order[0],
                "d": best_order[1],
                "q": best_order[2]
            }
            os.makedirs(params_dir, exist_ok=True)
            with open(params_file, "w") as f:
                json.dump(best_params, f, indent=4)
            print(f"Saved best parameters for {country} - {indicator} to {params_file}: {best_params}")

    # Train the final model using the best parameters (loaded or discovered)
    if best_params is not None:
        best_order = (best_params["p"], best_params["d"], best_params["q"])
        model = ARIMA(train_y, order=best_order)
        model_fit = model.fit()

        # Forecast using the final model
        best_predictions = model_fit.forecast(steps=len(test_y))

    # Save the predictions plot
    if best_predictions is not None:
        save_plot(train_y, test_y, best_predictions, country, indicator, model_name="ARIMA")

    return np.sqrt(mean_squared_error(test_y, best_predictions)), best_predictions


def train_prophet(train_df, test_y, country, indicator):
    params_dir = os.path.join("best_params", indicator)
    params_file = os.path.join(params_dir, f"Prophet_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")
        param_grid = {
            'changepoint_prior_scale': [0.01, 0.05, 0.1, 0.2, 0.5],
            'seasonality_mode': ['additive', 'multiplicative'],
            'changepoint_range': [0.8, 0.9, 1],  # Range of the history for changepoint detection
            'n_changepoints': [15, 20, 25, 30],  # Number of changepoints
            'yearly_seasonality': [True, False]  # Adding the toggle for yearly seasonality
        }

        best_rmse = float('inf')

        for changepoint_prior in param_grid['changepoint_prior_scale']:
            for seasonality_mode in param_grid['seasonality_mode']:
                for changepoint_range in param_grid['changepoint_range']:
                    for n_changepoints in param_grid['n_changepoints']:
                        for yearly_seasonality in param_grid['yearly_seasonality']:
                            try:
                                model = Prophet(
                                    yearly_seasonality=yearly_seasonality,
                                    weekly_seasonality=False,
                                    daily_seasonality=False,
                                    changepoint_prior_scale=changepoint_prior,
                                    seasonality_mode=seasonality_mode,
                                    changepoint_range=changepoint_range,
                                    n_changepoints=n_changepoints
                                )
                                model.fit(train_df)

                                future = model.make_future_dataframe(periods=len(test_y['y']), freq='Y')
                                forecast = model.predict(future)
                                predictions = forecast['yhat'].iloc[-len(test_y['y']):].values
                                rmse = np.sqrt(mean_squared_error(test_y['y'], predictions))

                                if rmse < best_rmse:
                                    best_rmse = rmse
                                    best_params = {
                                        'changepoint_prior_scale': changepoint_prior,
                                        'seasonality_mode': seasonality_mode,
                                        'changepoint_range': changepoint_range,
                                        'n_changepoints': n_changepoints,
                                        'yearly_seasonality': yearly_seasonality
                                    }
                            except Exception as e:
                                print(f"Error with parameters: changepoint_prior={changepoint_prior}, "
                                      f"seasonality_mode={seasonality_mode}, changepoint_range={changepoint_range}, "
                                      f"n_changepoints={n_changepoints}, yearly_seasonality={yearly_seasonality}. Error: {e}")
                                continue

        # Save the best parameters to JSON if found
        if best_params:
            os.makedirs(params_dir, exist_ok=True)
            with open(params_file, "w") as f:
                json.dump(best_params, f, indent=4)

    # Train the best model using loaded or selected parameters
    if best_params:
        best_model = Prophet(
            yearly_seasonality=best_params['yearly_seasonality'],
            weekly_seasonality=False,
            daily_seasonality=False,
            changepoint_prior_scale=best_params['changepoint_prior_scale'],
            seasonality_mode=best_params['seasonality_mode'],
            changepoint_range=best_params['changepoint_range'],
            n_changepoints=best_params['n_changepoints']
        )
        best_model.fit(train_df)
        future = best_model.make_future_dataframe(periods=len(test_y['y']), freq='Y')
        forecast = best_model.predict(future)
        predictions = forecast['yhat'].iloc[-len(test_y['y']):].values

        # Call the save_plot function to save the plot
        save_plot(train_df, test_y, predictions, country, indicator, model_name="Prophet")

        return np.sqrt(mean_squared_error(test_y['y'], predictions)), predictions
    else:
        print("No suitable parameters found.")
        return None, None

def train_holt_winters(train, test_y, country, indicator):
    params_dir = os.path.join("best_params", indicator)
    params_file = os.path.join(params_dir, f"Holt_Winters_{country}.json")
    best_params = None

    # Check if the best parameters JSON file exists
    if os.path.exists(params_file):
        with open(params_file, "r") as f:
            best_params = json.load(f)
        print(f"Loaded best parameters for {country} - {indicator} from {params_file}: {best_params}")
    else:
        # If not, run the grid search
        print(f"No pre-existing parameters for {country} - {indicator}. Running grid search.")
        param_grid = {
            'trend': [None, 'add', 'mul'],  # Focus on trend options
            'damped': [True, False],       # Damped trend
            'smoothing_level': [None, 0.1,0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9],  # Smoothing for level
            'smoothing_slope': [None, 0.1,0.2, 0.3,0.4, 0.5,0.6, 0.7,0.8, 0.9],  # Smoothing for trend
            'initialization_method': [None],
        }

        best_rmse = float('inf')
        for trend in param_grid['trend']:
            for damped in param_grid['damped']:
                for alpha in param_grid['smoothing_level']:
                    for beta in param_grid['smoothing_slope']:
                        for init_level in param_grid['initialization_method']:
                            try:
                                # Initialize model
                                model = ExponentialSmoothing(
                                    train,
                                    trend=trend,
                                    damped_trend=damped,
                                    seasonal=None
                                )
                                # Fit model with specific smoothing parameters
                                fitted_model = model.fit(
                                    smoothing_level=alpha,
                                    smoothing_slope=beta,
                                    initial_level=init_level,
                                    optimized=True
                                )
                                # Forecast and calculate RMSE
                                predictions = fitted_model.forecast(len(test_y))
                                rmse = np.sqrt(mean_squared_error(test_y, predictions))

                                # Track the best parameters
                                if rmse < best_rmse:
                                    best_rmse = rmse
                                    best_params = {
                                        "trend": trend,
                                        "damped_trend": damped,
                                        "smoothing_level": alpha,
                                        "smoothing_slope": beta,
                                        "initialization_method": init_level
                                    }
                            except Exception as e:
                                # Log errors for debugging
                                #print(f"Error with parameters: trend={trend}, damped={damped}, "
                                #      f"alpha={alpha}, beta={beta}. Error: {e}")
                                continue

        # Save the best parameters to JSON for future use
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)
        print(f"Saved best parameters for {country} - {indicator} to {params_file}: {best_params}")

    # Train the final model using the best parameters (loaded or discovered)
    best_model = ExponentialSmoothing(
        train,
        trend=best_params["trend"],
        damped_trend=best_params["damped_trend"]
    )
    best_fitted_model = best_model.fit(
        smoothing_level=best_params["smoothing_level"],
        smoothing_slope=best_params["smoothing_slope"],
        optimized=True
    )
    predictions = best_fitted_model.forecast(len(test_y))

    # Call the save_plot function to save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="Holt_Winters")

    print(f"Best Parameters used for {country} - {indicator}: {best_params}")
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions



def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)


def train_lstm(train, test_y, country, indicator, epochs=50, batch_size=16):
    # Define directories for saving/loading parameters
    params_dir = os.path.join("best_params", indicator)
    params_file = os.path.join(params_dir, f"LSTM_{country}.json")
    best_params = None

    # Check if the parameter file exists
    if os.path.exists(params_file):
        print(f"Loading parameters from {params_file}...")
        with open(params_file, "r") as f:
            best_params = json.load(f)
    else:
        print("No saved parameters found, performing grid search...")

        # Define the hyperparameters grid
        param_grid = {
            'lstm_units': [50, 100],
            'activation': ['relu'],
            'dropout_rate': [0.3, 0.5],
            'optimizer': ['adam'],
            'learning_rate': [0.001, 0.01],
            'sequence_length': [5, 10],
            'batch_size': [16, 32]
        }

        # Dictionary to map optimizer names to optimizer classes
        optimizer_dict = {
            'adam': Adam,
        }

        best_rmse = float('inf')
        best_predictions = None

        # Iterate through all possible hyperparameter combinations
        grid = ParameterGrid(param_grid)
        for params in grid:

            # Create sequences with the current sequence length
            seq_length = params['sequence_length']
            scaler = MinMaxScaler()
            train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
            test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

            X_train, y_train = create_sequences(train_scaled, seq_length)

            # Define LSTM model with current parameters
            model = Sequential([
                LSTM(params['lstm_units'], activation=params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
                Dropout(params['dropout_rate']),
                LSTM(params['lstm_units'], activation=params['activation']),
                Dropout(params['dropout_rate']),
                Dense(1)
            ])

            # Compile model with current optimizer and learning rate
            model.compile(optimizer=optimizer_dict[params['optimizer']](learning_rate=params['learning_rate']), loss='mse')
            early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

            model.fit(X_train, y_train, epochs=epochs, batch_size=params['batch_size'], verbose=0, callbacks=[early_stop])

            # Rolling prediction on test set
            predictions = []
            input_seq = train_scaled[-seq_length:].tolist()  # Start with last known sequence

            for _ in range(len(test_y)):
                X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
                y_pred = model.predict(X_input, verbose=0).flatten()[0]
                predictions.append(y_pred)
                input_seq.append([y_pred])

            # Convert predictions back to original scale
            predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

            # Calculate RMSE
            rmse = np.sqrt(mean_squared_error(test_y, predictions))

            # Track the best model and parameters
            if rmse < best_rmse:
                best_rmse = rmse
                best_params = params
                best_predictions = predictions

        # Save the best parameters to JSON
        os.makedirs(params_dir, exist_ok=True)
        with open(params_file, "w") as f:
            json.dump(best_params, f, indent=4)

    # Prepare data with the best sequence length

    optimizer_dict = {
            'adam': Adam,
        }
    seq_length = best_params['sequence_length']
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test_y.values.reshape(-1, 1))

    X_train, y_train = create_sequences(train_scaled, seq_length)

    # Define and compile the final model
    model = Sequential([
        LSTM(best_params['lstm_units'], activation=best_params['activation'], return_sequences=True, input_shape=(seq_length, 1)),
        Dropout(best_params['dropout_rate']),
        LSTM(best_params['lstm_units'], activation=best_params['activation']),
        Dropout(best_params['dropout_rate']),
        Dense(1)
    ])
    model.compile(optimizer=optimizer_dict[best_params['optimizer']](learning_rate=best_params['learning_rate']), loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(X_train, y_train, epochs=epochs, batch_size=best_params['batch_size'], verbose=0, callbacks=[early_stop])

    # Rolling prediction on the test set
    predictions = []
    input_seq = train_scaled[-seq_length:].tolist()

    for _ in range(len(test_y)):
        X_input = np.array(input_seq[-seq_length:]).reshape(1, seq_length, 1)
        y_pred = model.predict(X_input, verbose=0).flatten()[0]
        predictions.append(y_pred)
        input_seq.append([y_pred])

    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

    # Save the plot
    save_plot(train, test_y, predictions, country, indicator, model_name="LSTM")

    # Return RMSE and predictions
    return np.sqrt(mean_squared_error(test_y, predictions)), predictions


with open("countries.json", "r") as f:
    country_names = json.load(f)

with open("indicators.json", "r") as f:
    indicators = json.load(f)

data_folder = "data/base"
model_errors_rmse = {}
log_data = []
country_indicators_plots = {}
for country, country_code in country_names.items():
    for indicator, indicator_code in indicators.items():
        filename = f"{country.replace(' ', '_')}_{indicator.replace(' ', '_')}.parquet"
        filepath = os.path.join(data_folder, filename)
        
        if os.path.exists(filepath):
            df = pd.read_parquet(filepath)
            if 'Year' in df.columns and 'Value' in df.columns:
                df = df.set_index('Year').sort_index()
                df.index = pd.to_datetime(df.index, format='%Y')
                df = df.dropna()
                df = df.drop('Indicator', axis = 1)
                df_original = df.copy()
                
                for lag in range(1, 4):  
                    df[f'lag_{lag}'] = df['Value'].shift(lag)
                
                df['expanding_mean'] = df['Value'].expanding().mean()
                df['expanding_std'] = df['Value'].expanding().std()
                df['expanding_max'] = df['Value'].expanding().max()
                df['expanding_min'] = df['Value'].expanding().min()
                
                window_size = 3  
                df['rolling_mean_3'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std_3'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max_3'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min_3'] = df['Value'].rolling(window=window_size, min_periods=1).min()

                window_size = 5  
                df['rolling_mean_5'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std_5'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max_5'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min_5'] = df['Value'].rolling(window=window_size, min_periods=1).min()

                window_size = 4  
                df['rolling_mean_4'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std_4'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max_4'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min_4'] = df['Value'].rolling(window=window_size, min_periods=1).min()

                window_size = 2  
                df['rolling_mean_2'] = df['Value'].rolling(window=window_size, min_periods=1).mean()
                df['rolling_std_2'] = df['Value'].rolling(window=window_size, min_periods=1).std()
                df['rolling_max_2'] = df['Value'].rolling(window=window_size, min_periods=1).max()
                df['rolling_min_2'] = df['Value'].rolling(window=window_size, min_periods=1).min()
                
                
                #df = df.dropna()
                train_size = int(len(df) * 0.8)
                
                train_xgb, test_xgb = df.iloc[:train_size], df.iloc[train_size:]
                feature_columns = ['rolling_mean_2', 'rolling_std_2', 'rolling_max_2', 'rolling_min_2', 
                                   'rolling_mean_3', 'rolling_std_3', 'rolling_max_3', 'rolling_min_3',
                                   'rolling_mean_4', 'rolling_std_4', 'rolling_max_4', 'rolling_min_4',
                                   'rolling_mean_5', 'rolling_std_5', 'rolling_max_5', 'rolling_min_5', 
                                   'expanding_mean', 'expanding_std', 'expanding_max', 'expanding_min',
                                   'lag_1', 'lag_2', 'lag_3',]
                train_x, train_y = train_xgb[feature_columns], train_xgb['Value']
                test_x, test_y = test_xgb[feature_columns], test_xgb['Value']
                
                model_errors_rmse[(country, indicator)] = {}
                model_errors_rmse[(country, indicator)]['XGBoost'], xgb_pred = train_xgboost(train_x, train_y, test_x, test_y , country,indicator)
                model_errors_rmse[(country, indicator)]['ARIMA'],arime_pred = train_arima(df_original.iloc[:train_size]['Value'], 
                                                                               df_original.iloc[train_size:]['Value'], 
                                                                               country,indicator)
                model_errors_rmse[(country, indicator)]['Holt-Winters'] , es_pred = train_holt_winters(df_original.iloc[:train_size]['Value'], 
                                                                                             df_original.iloc[train_size:]['Value'],
                                                                                             country,indicator)
                model_errors_rmse[(country, indicator)]['LSTM'] , lstm_pred = train_lstm(df_original.iloc[:train_size]['Value'], 
                                                                             df_original.iloc[train_size:]['Value'],
                                                                             country,indicator)


                prophet_train_df = df_original.iloc[:train_size].reset_index().rename(columns={'Year': 'ds', 'Value': 'y'})
                prophet_test_df = df_original.iloc[train_size:].reset_index().rename(columns={'Year': 'ds', 'Value': 'y'})
                prophet_train_df['ds'] = pd.to_datetime(prophet_train_df['ds'])
                prophet_test_df['ds'] = pd.to_datetime(prophet_test_df['ds'])
                model_errors_rmse[(country, indicator)]['Prophet'] , prop_error = train_prophet(prophet_train_df, prophet_test_df,
                                                                                   country,indicator)
                
                sorted_models = sorted(model_errors_rmse[(country, indicator)].items(), key=lambda x: x[1])
                log_current_data = []
                for rank, (model_name, rmse) in enumerate(sorted_models, start=1):
                    log_data.append([country, indicator, model_name, rmse, rank])
                    log_current_data.append([country, indicator, model_name, rmse, rank])

                
                model_ranks = {(entry[0], entry[1], entry[2]): entry[4] for entry in log_current_data}
                plt.figure(figsize=(10, 6))
                plt.plot(df_original.iloc[:train_size].index, df_original.iloc[:train_size]['Value'], 
                         label='Train Data', color='green', linestyle='--')
                plt.plot(df_original.iloc[train_size:].index, df_original.iloc[train_size:]['Value'],
                          label='Test Data', color='red', linestyle='--')
                
                plt.plot(df_original.iloc[train_size:].index, arime_pred, 
                         label=f'ARIMA ({model_ranks.get((country, indicator, "ARIMA"), "N/A")})', 
                         color='blue', linestyle='-', marker='o')
                
                plt.plot(df_original.iloc[train_size:].index, es_pred, 
                          label=f'Holt-Winters ({model_ranks.get((country, indicator, "Holt-Winters"), "N/A")})',
                         color='yellow',linestyle='-', marker='o')
                
                plt.plot(df_original.iloc[train_size:].index, lstm_pred, 
                         label=f'LSTM ({model_ranks.get((country, indicator, "LSTM"), "N/A")})', 
                         color='black', linestyle='-', marker='o')
                
                plt.plot(df_original.iloc[train_size:].index, xgb_pred, 
                         label=f'XGBoost ({model_ranks.get((country, indicator, "XGBoost"), "N/A")})', 
                         color='pink', linestyle='-', marker='o')
                
                plt.plot(df_original.iloc[train_size:].index, prop_error, 
                         label=f'Prophet ({model_ranks.get((country, indicator, "Prophet"), "N/A")})', 
                         color='brown', linestyle='-', marker='o')
                
                
                plt.title(f'Predicted vs Actual for {country} - {indicator}')
                plt.xlabel('Year')
                plt.ylabel('Value')
                plt.legend()

                # Create subfolder for the indicator if it doesn't exist
                indicator_folder = os.path.join('images', 'model_plot', 'Indicators', indicator)
                os.makedirs(indicator_folder, exist_ok=True)
                
                # Save the plot in the Indicators folder with dynamic model name
                plot_filename_indicator = os.path.join(indicator_folder, f'AllModels_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
                plt.savefig(plot_filename_indicator)

                # Create subfolder for the country if it doesn't exist
                country_folder = os.path.join('images', 'model_plot', 'Countries', country)
                os.makedirs(country_folder, exist_ok=True)
                
                # Save the same plot in the Countries folder with dynamic model name
                plot_filename_country = os.path.join(country_folder, f'AllModels_{country.replace(" ", "_")}_{indicator.replace(" ", "_")}.png')
                plt.savefig(plot_filename_country)

                
                #plt.savefig(f'Predicted vs Actual for {country} - {indicator}')

                if country not in country_indicators_plots:
                    country_indicators_plots[country] = []
                country_indicators_plots[country].append(plt.gcf())
                plt.close()

# After collecting all plots for each country, create a combined plot
for country, plots in country_indicators_plots.items():
    n_plots = len(plots)
    n_cols = 2  # Set number of columns in the grid
    n_rows = (n_plots + 1) // n_cols  # Calculate required number of rows
    
    plt.figure(figsize=(15, 5 * n_rows))  # Adjust figure size for grid layout
    for i, plot in enumerate(plots, start=1):
        plt.subplot(n_rows, n_cols, i)
        
        # Copy each plot's data by extracting from the original plot and plotting again
        for ax in plot.get_axes():  # Iterate through all axes in the current plot
            for line in ax.get_lines():  # Get lines (or other elements) from the original plot
                plt.plot(line.get_xdata(), line.get_ydata(), label=line.get_label(), color=line.get_color(), linestyle=line.get_linestyle(), marker=line.get_marker())
        
        indicator_name = list(indicators.keys())[i - 1]
        plt.title(f'{country} - {indicator_name}')
        plt.xlabel('Year')
        plt.ylabel('Value')
        plt.legend()

    # Save the combined plot
    country_folder = os.path.join('images', 'model_plot', 'Countries', country)
    os.makedirs(country_folder, exist_ok=True)
    plot_filename_country = os.path.join(country_folder, f'{country.replace(" ", "_")}_all_indicators.png')
    plt.tight_layout()
    plt.savefig(plot_filename_country)
    plt.close()

# Create a dictionary to store all indicator plots for later use
indicator_plots = {indicator: [] for indicator in indicators.keys()}

# After collecting all plots for each country, create a combined plot for each indicator
for country, plots in country_indicators_plots.items():
    for i, plot in enumerate(plots, start=1):
        indicator_name = list(indicators.keys())[i - 1]
        
        # Append each plot to the corresponding indicator's list
        indicator_plots[indicator_name].append(plot)

# Now create a combined plot for all countries for each indicator
for indicator, plots in indicator_plots.items():
    n_plots = len(plots)
    n_cols = 2  # Set number of columns in the grid
    n_rows = (n_plots + 1) // n_cols  # Calculate required number of rows
    
    plt.figure(figsize=(15, 5 * n_rows))  # Adjust figure size for grid layout
    for i, plot in enumerate(plots, start=1):
        plt.subplot(n_rows, n_cols, i)
        
        # Copy each plot's data by extracting from the original plot and plotting again
        for ax in plot.get_axes():  # Iterate through all axes in the current plot
            for line in ax.get_lines():  # Get lines (or other elements) from the original plot
                plt.plot(line.get_xdata(), line.get_ydata(), label=line.get_label(), color=line.get_color(), linestyle=line.get_linestyle(), marker=line.get_marker())
        
        plt.title(f'{indicator} - {list(country_names)[i-1]}')
        plt.xlabel('Year')
        plt.ylabel('Value')
        plt.legend()

    # Save the combined plot for the indicator across all countries
    indicator_folder = os.path.join('images', 'model_plot', 'Indicators', indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    plot_filename_indicator = os.path.join(indicator_folder, f'All_Countries_{indicator.replace(" ", "_")}.png')
    plt.tight_layout()
    plt.savefig(plot_filename_indicator)
    plt.close()

from datetime import datetime

log_df = pd.DataFrame(log_data, columns=['Country', 'Indicator', 'Model', 'RMSE', 'Rank'])

# Generate timestamp
timestamp = datetime.now().strftime("%Y-%m-%d--%H-%M")

# Define base folder
base_folder = "data/final"

# Ensure subfolders exist
subfolders = {
    "log": "log_files",
    "country_rankings": "country_rankings",
    "indicator_rankings": "indicator_rankings",
    "overall_rankings": "overall_rankings"
}

for subfolder in subfolders.values():
    os.makedirs(os.path.join(base_folder, subfolder), exist_ok=True)

# Save the log DataFrame
log_file_path = os.path.join(base_folder, subfolders["log"], f"model_error_log_{timestamp}.csv")
log_df.to_csv(log_file_path, index=False)
print(f"Log file saved to {log_file_path}")

# Create the pivot DataFrame for country rankings
pivot_country_df = log_df.pivot_table(index=['Country', 'Model'], 
                                      columns='Rank', 
                                      aggfunc='size', 
                                      fill_value=0)

# Rename columns to reflect the rank (1, 2, 3, etc.)
pivot_country_df.columns = ['1', '2', '3', '4', '5']

# Reset index for clear visibility
pivot_country_df.reset_index(inplace=True)
country_file_path = os.path.join(base_folder, subfolders["country_rankings"], f"model_rankings_by_country_{timestamp}.csv")
pivot_country_df.to_csv(country_file_path, index=False)
print(f"Country rankings saved to {country_file_path}")

# Create the pivot DataFrame for indicator rankings
pivot_df = log_df.pivot_table(index=['Indicator', 'Model'], 
                              columns='Rank', 
                              aggfunc='size', 
                              fill_value=0)

# Rename columns to reflect the rank (1, 2, 3, etc.)
pivot_df.columns = ['1', '2', '3', '4', '5']

# Reset index for clear visibility
pivot_df.reset_index(inplace=True)
indicator_file_path = os.path.join(base_folder, subfolders["indicator_rankings"], f"model_rankings_by_indicator_{timestamp}.csv")
pivot_df.to_csv(indicator_file_path, index=False)
print(f"Indicator rankings saved to {indicator_file_path}")

# Create the overall pivot DataFrame for model rankings
overall_df = log_df.pivot_table(index=['Model'], 
                                columns='Rank', 
                                aggfunc='size', 
                                fill_value=0)

# Rename columns to reflect the rank (1, 2, 3, etc.)
overall_df.columns = ['1', '2', '3', '4', '5']

# Reset index for clear visibility
overall_df.reset_index(inplace=True)
overall_file_path = os.path.join(base_folder, subfolders["overall_rankings"], f"model_rankings_overall_{timestamp}.csv")
overall_df.to_csv(overall_file_path, index=False)
print(f"Overall rankings saved to {overall_file_path}")


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading parameters from best_params\GDP per Capita (USD)\XGBoost_Czech Republic.json...
Loaded best parameters for Czech Republic - GDP per Capita (USD) from best_params\GDP per Capita (USD)\ARIMA_Czech Republic.json: {'p': 12, 'd': 2, 'q': 6}


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Loaded best parameters for Czech Republic - GDP per Capita (USD) from best_params\GDP per Capita (USD)\Holt_Winters_Czech Republic.json: {'trend': 'mul', 'damped_trend': False, 'smoothing_level': 0.5, 'smoothing_slope': 0.1, 'initialization_method': None}


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency YS-JAN will be used.
  self._init_dates(dates, freq)
C:\Users\admin\AppData\Local\Temp\ipykernel_2388\506649542.py:376: FutureWarning: the 'smoothing_slope' keyword is deprecated, use 'smoothing_trend' instead.
  best_fitted_model = best_model.fit(
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


...

Best Parameters used for Italy - Exports of goods and services (% of GDP): {'trend': 'add', 'damped_trend': False, 'smoothing_level': 0.6, 'smoothing_slope': 0.3, 'initialization_method': None}
Loading parameters from best_params\Exports of goods and services (% of GDP)\LSTM_Italy.json...


c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)
22:26:16 - cmdstanpy - INFO - Chain [1] start processing


Loading parameters from best_params\Exports of goods and services (% of GDP)\Prophet_Italy.json...


22:26:17 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\admin\Desktop\diplomovka\Code\Model_staff\myenv\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  dates = pd.date_range(


Log file saved to data/final\log_files\model_error_log_2025-02-17--22-26.csv
Country rankings saved to data/final\country_rankings\model_rankings_by_country_2025-02-17--22-26.csv
Indicator rankings saved to data/final\indicator_rankings\model_rankings_by_indicator_2025-02-17--22-26.csv
Overall rankings saved to data/final\overall_rankings\model_rankings_overall_2025-02-17--22-26.csv


### TESTS

In [ ]:

"""
# Create the pivot DataFrame, where rows are indicator-model pairs, and columns are the ranks (1, 2, 3)
pivot_country_df = log_df.pivot_table(index=['Country', 'Model'], 
                              columns='Rank', 
                              aggfunc='size', 
                              fill_value=0)

# Rename columns to reflect the rank (1, 2, 3)
pivot_country_df.columns = ['1', '2', '3','4','5']

# Reset index for clear visibility
pivot_country_df.reset_index(inplace=True)
#pivot_country_df.to_csv("model_rankings_by_country_with_models.csv", index=False)
"""
pivot_country_df

,Country,Model,1,2,3,4,5
0,Austria,ARIMA,3,4,0,0,0
1,Austria,Holt-Winters,1,2,3,1,0
2,Austria,LSTM,0,0,1,1,5
3,Austria,Prophet,0,1,3,1,2
4,Austria,XGBoost,3,0,0,4,0
5,Czech Republic,ARIMA,4,3,0,0,0
6,Czech Republic,Holt-Winters,0,2,5,0,0
7,Czech Republic,LSTM,0,0,0,1,6
8,Czech Republic,Prophet,1,0,2,4,0
9,Czech Republic,XGBoost,2,2,0,2,1


In [ ]:
"""
# Create the pivot DataFrame, where rows are indicator-model pairs, and columns are the ranks (1, 2, 3)
pivot_df = log_df.pivot_table(index=['Indicator', 'Model'], 
                              columns='Rank', 
                              aggfunc='size', 
                              fill_value=0)

# Rename columns to reflect the rank (1, 2, 3)
pivot_df.columns = ['1', '2', '3','4','5']

# Reset index for clear visibility
pivot_df.reset_index(inplace=True)
#pivot_df.to_csv("model_rankings_by_indicator_with_models.csv", index=False)
"""
pivot_df

,Indicator,Model,1,2,3,4,5
0,Exports of goods and services (% of GDP),ARIMA,5,2,1,0,0
1,Exports of goods and services (% of GDP),Holt-Winters,0,2,4,2,0
2,Exports of goods and services (% of GDP),LSTM,0,0,1,2,5
3,Exports of goods and services (% of GDP),Prophet,2,2,2,1,1
4,Exports of goods and services (% of GDP),XGBoost,1,2,0,3,2
5,GDP (USD),ARIMA,3,4,1,0,0
6,GDP (USD),Holt-Winters,4,3,1,0,0
7,GDP (USD),LSTM,0,0,0,3,5
8,GDP (USD),Prophet,0,0,6,2,0
9,GDP (USD),XGBoost,1,1,0,3,3


In [ ]:
"""
# Create the pivot DataFrame, where rows are indicator-model pairs, and columns are the ranks (1, 2, 3)
overall_df = log_df.pivot_table(index=['Model'], 
                              columns='Rank', 
                              aggfunc='size', 
                              fill_value=0)

# Rename columns to reflect the rank (1, 2, 3)
overall_df.columns = ['1', '2', '3','4','5']

# Reset index for clear visibility
overall_df.reset_index(inplace=True)
#overall_df.to_csv("model_rankings_overall_placements.csv", index=False)
"""
overall_df

,Model,1,2,3,4,5
0,ARIMA,25,26,5,0,0
1,Holt-Winters,10,16,26,4,0
2,LSTM,0,0,2,16,38
3,Prophet,2,6,22,20,6
4,XGBoost,19,8,1,16,12
